# Librerias Necesarias

In [1]:
#!pip install umap
#!pip install mlxtend
#!!pip install 'pami[gpu]'

In [2]:
import gc
import matplotlib.pyplot as plt
import mlxtend.frequent_patterns as FP
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import shap
from statsmodels.stats.weightstats import DescrStatsW
import warnings

In [3]:
import sys
sys.setrecursionlimit(16385)

In [4]:
#Libreamos espacio en la GPU
gc.collect()

0

In [5]:
#Para visualizar todas las columnas
pd.set_option('display.max_columns', None)

In [6]:
#Eliminar las advertencias 
warnings.filterwarnings("ignore")

# Lectura de Datos 
Cargamos los datasets resultantes

In [7]:
base_path = "./ConjuntosDatos/"
endutih_path = base_path + "ENDUTIH2020/conjuntos_de_datos/"

In [9]:
endutih_vivhogar = pd.read_csv(endutih_path+"viv_hog_agrupados.csv",  dtype={"ESTRATO":str, "Grupo":str})
endutih_res = pd.read_csv(endutih_path+"residente_agrupados.csv",  dtype={"ESTRATO":str, "Grupo":str})
endutih_usr = pd.read_csv(endutih_path+"usuario_agrupados.csv", low_memory=False,  dtype={"ESTRATO":str, "Grupo":str})
endutih_usr2 = pd.read_csv(endutih_path+"usuario2_agrupados.csv", low_memory=False,  dtype={"ESTRATO":str, "Grupo":str})
estados = pd.read_csv(base_path+"CodigoEdos.csv")
columnas_sociotec = pd.read_csv("./columnas_clustering/cols_clus_medio_bajo.csv").columns

endutih_completa = pd.merge(endutih_vivhogar,endutih_res.drop(["UPM_DIS","ESTRATO","ENT","DOMINIO", "EST_DIS", "TLOC","Grupo"], axis=1), on = ["UPM","VIV_SEL", "HOGAR"])
endutih_completa = pd.merge(endutih_completa, estados["Codigo ENT".split()], left_on=["ENT"], right_on="Codigo", suffixes=("_orig","")).drop("Codigo ENT_orig".split(), axis=1)

In [10]:
endutih_completa["llave_tmp"] = endutih_completa["ESTRATO"]+endutih_completa["DOMINIO"]+endutih_completa["Grupo"]+endutih_completa["ENT"]

In [11]:
endutih_completa.head()

,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,HOGAR,VIV_SEL,NUM_REN,PAREN,SEXO,EDAD,DIA,MES,P3_7,NIVEL,GRADO,P3_9_1,P3_9_2,P3_9_3,P3_10,P3_11,P3_12,FAC_HOGAR,ENT,llave_tmp
0,2,1,1,1,1,2,2,2,1,1.0,572,2,1,1,2,2,1,2.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,575,U,1,1,11,206264,570,17,1,1,1,1,2,54,27.0,3.0,2.0,0.0,0.0,9.0,9.0,1.0,6.0,4.0,NaN,575,B.C.,1U11B.C.
1,2,1,1,1,1,2,2,2,1,1.0,572,2,1,1,2,2,1,2.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,575,U,1,1,11,206264,570,17,1,1,2,3,2,22,7.0,1.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,NaN,1.0,575,B.C.,1U11B.C.
2,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,U,1,1,11,206264,570,17,1,2,1,1,2,36,29.0,6.0,2.0,3.0,3.0,1.0,1.0,1.0,1.0,NaN,1.0,575,B.C.,1U11B.C.
3,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,U,1,1,11,206264,570,17,1,2,2,3,1,20,29.0,3.0,2.0,6.0,2.0,1.0,1.0,1.0,1.0,NaN,1.0,575,B.C.,1U11B.C.
4,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,U,1,1,11,206264,570,17,1,2,3,3,2,19,20.0,2.0,2.0,6.0,3.0,1.0,1.0,1.0,1.0,NaN,1.0,575,B.C.,1U11B.C.


In [12]:
resumen_idtmex = pd.read_csv(base_path+"long_IDTMex_desglosado_dominio_grupos_edo.csv", index_col=0, dtype={"ESTRATO":str, "Grupo":str})
resumen_idtmex

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR
0,2,R,20,Ags.,0.526023,0.418287,0.463184,4.703610,10739686
1,2,R,20,B.C.,0.512853,0.447157,0.431790,4.703621,10739686
2,2,R,20,B.C.S,0.545126,0.487307,0.492930,5.115594,10739686
3,2,R,20,Coah.,0.333664,0.256183,0.388742,3.136870,10739686
4,2,R,20,Col.,0.486668,0.414325,0.435024,4.474019,10739686
...,...,...,...,...,...,...,...,...,...
515,1,U,11,Tab.,0.343698,0.241601,0.415164,3.171525,5900321
516,1,U,11,Tamps.,0.527802,0.464130,0.415717,4.799163,5900321
517,1,U,11,Tlax.,0.382584,0.252696,0.353048,3.247217,5900321
518,1,U,11,Ver.,0.311895,0.189587,0.319908,2.645744,5900321


In [13]:
resumen_idtmex.describe(include="all")

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR
count,520,520,520,520,520.000000,520.000000,520.000000,520.000000,5.200000e+02
unique,4,2,15,32,NaN,NaN,NaN,NaN,NaN
top,2,U,20,B.C.S,NaN,NaN,NaN,NaN,NaN
freq,192,418,62,20,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,0.683853,0.565991,0.618888,6.237152,7.608668e+06
std,NaN,NaN,NaN,NaN,0.212327,0.207891,0.191084,1.849581,1.357665e+07
min,NaN,NaN,NaN,NaN,0.123636,0.000000,0.166667,1.511805,2.505400e+04
25%,NaN,NaN,NaN,NaN,0.510543,0.445805,0.463121,4.934255,2.286600e+05
50%,NaN,NaN,NaN,NaN,0.701144,0.598388,0.596687,6.567132,3.791910e+05
75%,NaN,NaN,NaN,NaN,0.856716,0.668162,0.777778,7.666667,1.039782e+07


# Estratificación de resultados

In [14]:
np.linspace(1.51,10,5)

array([ 1.51  ,  3.6325,  5.755 ,  7.8775, 10.    ])

In [15]:
condiciones = [(resumen_idtmex.IDTMex<=3.63), (resumen_idtmex.IDTMex<=5.76), (resumen_idtmex.IDTMex<=7.88), (resumen_idtmex.IDTMex>7.88)]
etiquetas = "Bajo Medio-bajo Medio-alto Alto".split()

resumen_idtmex["clasificacion"] = np.select(condiciones, etiquetas)
resumen_idtmex.head()

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR,clasificacion
0,2,R,20,Ags.,0.526023,0.418287,0.463184,4.703610,10739686,Medio-bajo
1,2,R,20,B.C.,0.512853,0.447157,0.431790,4.703621,10739686,Medio-bajo
2,2,R,20,B.C.S,0.545126,0.487307,0.492930,5.115594,10739686,Medio-bajo
3,2,R,20,Coah.,0.333664,0.256183,0.388742,3.136870,10739686,Bajo
4,2,R,20,Col.,0.486668,0.414325,0.435024,4.474019,10739686,Medio-bajo


In [17]:
resumen_grupos = resumen_idtmex.groupby("clasificacion")["FAC_HOGAR IDTMex".split()].agg(
    prom=("IDTMex",lambda x: np.average(x, weights=resumen_idtmex.loc[x.index, "FAC_HOGAR"])),
    var=("IDTMex",lambda x:  DescrStatsW(x, weights=resumen_idtmex.loc[x.index, "FAC_HOGAR"], ddof=0).var),
    tot=( "FAC_HOGAR","sum")
)
resumen_grupos["perc"] = 100*resumen_grupos['tot']/resumen_grupos["tot"].sum()
resumen_grupos.loc[etiquetas]

,prom,var,tot,perc
clasificacion,,,,
Bajo,2.890767,0.158735,651646383,16.470243
Medio-bajo,4.891165,0.248386,1860137576,47.014637
Medio-alto,6.692654,0.306119,1335225681,33.747585
Alto,8.324801,0.117679,109497724,2.767535


In [18]:
x = resumen_idtmex.IDTMex
prom = np.average(x, weights=resumen_idtmex["FAC_HOGAR"])
var = DescrStatsW(x, weights=resumen_idtmex["FAC_HOGAR"], ddof=0).var
prom, var

(5.264680807666978, 2.1905641407022207)

# Seleccion de grupos a analizar
Se sleccionan los grupos que se encuentran en el límite para subir la clasificación

In [20]:
grupos_bajos = resumen_idtmex[resumen_idtmex.clasificacion=="Bajo"]
grupos_bajos["llave_tmp"] = grupos_bajos["ESTRATO"]+grupos_bajos["DOMINIO"]+grupos_bajos["Grupo"]+grupos_bajos["ENT"]
grupos_bajos = grupos_bajos[grupos_bajos["IDTMex"] >= grupos_bajos["IDTMex"].quantile(0.9)]
grupos_bajos

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR,clasificacion,llave_tmp
9,2,R,20,Gto.,0.385818,0.295996,0.400745,3.528747,10739686,Bajo,2R20Gto.
20,2,R,20,Q. Roo.,0.352222,0.299472,0.398243,3.403262,10739686,Bajo,2R20Q. Roo.
462,1,R,11,B.C.,0.370303,0.270911,0.423337,3.411529,21166834,Bajo,1R11B.C.
470,1,R,11,Gto.,0.403872,0.319342,0.364866,3.622589,21166834,Bajo,1R11Gto.
481,1,R,11,Qro.,0.377838,0.279374,0.372411,3.373671,21166834,Bajo,1R11Qro.
519,1,U,11,Yuc.,0.368952,0.278938,0.391396,3.374351,5900321,Bajo,1U11Yuc.


In [21]:
grupos_bajos.describe()

,acceso,uso,aptitudes,IDTMex,FAC_HOGAR
count,6.000000,6.000000,6.000000,6.000000,6.000000e+00
mean,0.376501,0.290672,0.391833,3.452358,1.514670e+07
std,0.017451,0.017797,0.021060,0.101161,6.827373e+06
min,0.352222,0.270911,0.364866,3.373671,5.900321e+06
25%,0.369290,0.279047,0.377157,3.381579,1.073969e+07
50%,0.374071,0.287685,0.394820,3.407395,1.595326e+07
75%,0.383823,0.298603,0.400120,3.499442,2.116683e+07
max,0.403872,0.319342,0.423337,3.622589,2.116683e+07


In [23]:
casos_bajos = endutih_completa[endutih_completa["llave_tmp"].isin(grupos_bajos["llave_tmp"])]
casos_bajos = pd.merge(casos_bajos, estados["Codigo ENT".split()], on = "ENT") 
casos_bajos

,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,HOGAR,VIV_SEL,NUM_REN,PAREN,SEXO,EDAD,DIA,MES,P3_7,NIVEL,GRADO,P3_9_1,P3_9_2,P3_9_3,P3_10,P3_11,P3_12,FAC_HOGAR,ENT,llave_tmp,Codigo
0,2,2,2,1,2,2,2,4,1,1.0,699,1,1,2,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,699,R,4,1,11,260010,631,15,1,1,1,1,1,42,28.0,7.0,2.0,2.0,6.0,2.0,2.0,1.0,1.0,NaN,3.0,699,B.C.,1R11B.C.,2
1,2,2,2,1,2,2,2,4,1,1.0,699,1,1,2,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,699,R,4,1,11,260010,631,15,1,1,2,2,2,37,9.0,12.0,2.0,2.0,6.0,2.0,1.0,1.0,6.0,1.0,4.0,699,B.C.,1R11B.C.,2
2,2,2,2,1,2,2,2,4,1,1.0,699,1,1,2,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,699,R,4,1,11,260010,631,15,1,1,3,3,2,15,18.0,2.0,1.0,3.0,3.0,2.0,1.0,1.0,5.0,4.0,NaN,699,B.C.,1R11B.C.,2
3,2,2,2,1,2,2,2,4,1,1.0,699,1,1,2,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,699,R,4,1,11,260010,631,15,1,1,4,3,1,7,17.0,11.0,1.0,2.0,2.0,2.0,1.0,1.0,NaN,NaN,NaN,699,B.C.,1R11B.C.,2
4,2,1,2,1,1,1,1,5,1,1.0,699,2,2,2,1,2,1,4.0,4.0,2,2,2,1.0,1,1.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,1.0,2,1,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,389.0,2.0,3.0,3.0,699,R,4,1,11,260010,631,15,1,2,1,1,1,32,7.0,11.0,2.0,6.0,3.0,2.0,1.0,1.0,1.0,NaN,1.0,699,B.C.,1R11B.C.,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4909,2,1,2,1,1,1,2,3,1,1.0,306,2,1,2,2,2,2,0.0,0.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306,U,3,1,11,3104022,8038,321,1,17,3,5,2,15,7.0,8.0,2.0,3.0,3.0,2.0,2.0,2.0,6.0,4.0,NaN,306,Yuc.,1U11Yuc.,31
4910,3,1,2,1,1,1,2,4,1,1.0,306,2,2,2,1,2,1,2.0,2.0,2,2,1,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,2.0,1.0,2.0,2.0,0.0,2,2,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,209.0,2.0,0.0,2.0,0.0,3.0,2.0,3.0,306,U,3,1,11,3104022,8038,321,1,20,1,1,1,45,5.0,5.0,2.0,3.0,3.0,2.0,2.0,1.0,1.0,NaN,2.0,306,Yuc.,1U11Yuc.,31
4911,3,1,2,1,1,1,2,4,1,1.0,306,2,2,2,1,2,1,2.0,2.0,2,2,1,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,2.0,1.0,2.0,2.0,0.0,2,2,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,209.0,2.0,0.0,2.0,0.0,3.0,2.0,3.0,306,U,3,1,11,3104022,8038,321,1,20,2,2,2,39,24.0,12.0,2.0,3.0,3.0,2.0,2.0,1.0,1.0,NaN,1.0,306,Yuc.,1U11Yuc.,31
4912,3,1,2,1,1,1,2,4,1,1.0,306,2,2,2,1,2,1,2.0,2.0,2,2,1,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,2.0,1.0,2.0,2.0,0.0,2,2,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,209.0,2.0,0.0,2.0,0.0,3.0,2.0,3.0,306,U,3,1,11,3104022,8038,321,1,20,3,3,1,10

In [25]:
casos_bajos["DOMINIO Grupo Codigo".split()].drop_duplicates().to_csv("./ConjuntosDatos/casos_limite.csv", index=False)

# Estados con un valor alto

In [29]:
edos_mejores = resumen_idtmex[resumen_idtmex["ENT"].isin("C.D.Mx|N.L|Edo.Méx.|B.C.|Son.|Jal.".split("|"))]
edos_mejores

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR,clasificacion
1,2,R,20,B.C.,0.512853,0.447157,0.431790,4.703621,10739686,Medio-bajo
7,2,R,20,C.D.Mx,0.633335,0.635994,0.549197,6.175713,10739686,Medio-alto
11,2,R,20,Jal.,0.501713,0.394972,0.461210,4.509161,10739686,Medio-bajo
23,2,R,20,Son.,0.495414,0.506602,0.411194,4.830451,10739686,Medio-bajo
39,2,R,22,B.C.,0.750000,0.888889,0.500000,7.555556,46226,Medio-alto
...,...,...,...,...,...,...,...,...,...,...
485,1,R,11,Son.,0.397976,0.381334,0.417953,3.953149,21166834,Medio-bajo
492,1,U,11,B.C.,0.600000,0.446377,0.430409,5.046326,5900321,Medio-bajo
498,1,U,11,C.D.Mx,0.500000,0.502778,0.450980,4.913072,5900321,Medio-bajo
503,1,U,11,Jal.,0.300000,0.222222,0.366667,2.822222,5900321,Bajo


In [32]:
for ent in edos_mejores["ENT"].unique():
    print(ent)
    print(edos_mejores[edos_mejores["ENT"]==ent].describe())
    print(edos_mejores.loc[edos_mejores["ENT"]==ent,"clasificacion"].value_counts())
    print("-"*20)

B.C.
          acceso        uso  aptitudes     IDTMex     FAC_HOGAR
count  18.000000  18.000000  18.000000  18.000000  1.800000e+01
mean    0.759033   0.708130   0.615872   7.100396  7.040302e+06
std     0.182459   0.185973   0.176297   1.675368  1.345526e+07
min     0.370303   0.270911   0.286306   3.411529  4.622600e+04
25%     0.611219   0.642323   0.490639   6.124496  2.302370e+05
50%     0.757622   0.777197   0.600629   7.791346  3.695385e+05
75%     0.914762   0.813413   0.729189   8.028125  9.273448e+06
max     1.000000   1.000000   0.900093   9.466667  5.211225e+07
Alto          8
Medio-alto    6
Medio-bajo    3
Bajo          1
Name: clasificacion, dtype: int64
--------------------
C.D.Mx
          acceso        uso  aptitudes     IDTMex     FAC_HOGAR
count  16.000000  16.000000  16.000000  16.000000  1.600000e+01
mean    0.758381   0.751932   0.696387   7.434027  6.594524e+06
std     0.164228   0.169520   0.120339   1.401423  1.372612e+07
min     0.500000   0.333333   0.45098

In [36]:
estados[estados["ENT"].isin("C.D.Mx|N.L|Edo.Méx.|B.C.|Son.|Jal.".split("|"))]

,Codigo,Nombre,ENT
1,2,Baja California,B.C.
8,9,Ciudad de México,C.D.Mx
13,14,Jalisco,Jal.
25,26,Sonora,Son.
